In [1]:
class SimpleList:
    def __init__(self, items):
        self._items = list(items)

    def add(self, item):
        self._items.append(item)

    # This makes indexing and iteration possible
    def __getitem__(self, index):
        return self._items[index]

    def sort(self):
        self._items.sort()

    def __len__(self):
        return len(self._items)

    def __repr__(self):
        return f'{type(self).__name__}({self._items!r})'

# If we want to have every functionality of base class and then further extend the functionality
# we will have to first call super().__func__ and then extend the function
class SortedList(SimpleList):
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()

    def add(self, item):
        super().add(item)
        self.sort()

In [2]:
sl = SortedList([25, 3, 5, 90, 76])
sl

SortedList([3, 5, 25, 76, 90])

In [3]:
sl.add(-40)
sl

SortedList([-40, 3, 5, 25, 76, 90])

In [4]:
class mutation(list):
    def append(self, item):
        super().append(item)
        self.sort()
a = mutation([1,2,4,2,7,5])
a

[1, 2, 4, 2, 7, 5]

In [5]:
# I changed the append method of list, not recommended
# now every time we append something in list, it is sorted
a.append(4)
a

[1, 2, 2, 4, 4, 5, 7]

<b> Checking class base </b>


In [6]:
print(isinstance(sl, SortedList))
print(isinstance(sl, SimpleList))
print(isinstance(sl, list))
print(isinstance(sl, (list, int, SortedList)))

True
True
False
True


In [7]:
class MyInt(int):
    pass
class MySpecialInt(MyInt):
    pass

print(issubclass(MySpecialInt, MyInt))
print(issubclass(MySpecialInt, int))

True
True


In [8]:
class IntList(SimpleList):
    def __init__(self, items=()):
        for x in items: self._validate(x)
        super().__init__(items)

    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only supports integer values.')

    def add(self, item):
        self._validate(item)
        super().add(item)


# Multiple Inheritence
class SortedIntList(IntList, SortedList):
    pass

In [9]:
sil = SortedIntList([2,8,3,5])
sil
# List is sorted

SortedIntList([2, 3, 5, 8])

In [10]:
try:
    sil_invalid = SortedIntList([1,2,3,'4'])
except TypeError as e:
    print(e)
# Int validation is also working fine

IntList only supports integer values.


In [11]:
sil.add(6)
sil
# Add method of sorted list working

SortedIntList([2, 3, 5, 6, 8])

In [12]:
try:
    sil.add('7')
except TypeError as e:
    print(e)
# Add method of IntList working

IntList only supports integer values.


What is the order in which methods of different base classes are called?<br>
We will study about this in method resolution order.

In [13]:
# Base class __init__
class Base1:
    def __init__(self):
        print('Base1.__init__')
class Base2:
    def __init__(self):
        print('Base2.__init__')
        
class Sub(Base2, Base1):
    pass

sub = Sub()

Base2.__init__


** Initilaizer of only first base class is called, we will learn how to call both initializer using super() later<br>
Also we have to inspect why both base class __ init__ worked correcty in case of SortedIntList

In [14]:
# Easy way to get base classes and there orders
Sub.__bases__

(__main__.Base2, __main__.Base1)

### Method Resolution Order (MRO)
Ordering of an inheritence graph which determines which implementation to use when invoking a method.

In [15]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [16]:
class A:
    def func(self):
        return 'A.func'

class B(A):
    def func(self):
        return 'B.func'

class C(A):
    def func(self):
        return 'C.func'

class D(C, B):
    pass

In [17]:
# Python will start looking for called method in same order till it finds it.
D.__mro__

(__main__.D, __main__.C, __main__.B, __main__.A, object)

In [18]:
C.__mro__

(__main__.C, __main__.A, object)

<b> Why was sortedIntList both sorted and validated???</b><br>
The answer lies in how super() works

In [19]:
# MRO creation uses C3, which works on folowing priciples
# Subclasses will always come before base classes
# Base class order from class definition is preserved

try:
    class A: pass
    class B(A): pass
    class C(A): pass
    class D(B, A, C): pass
except Exception as e:
    print(e)
    
# According to first rule C should always come before A, according to second rule of class definition, C should come aftter A
# As there is a conflict, python will not allow this inheritence

Cannot create a consistent method resolution
order (MRO) for bases A, C


### super()
Given an MRO and a class C in that MRO, super() gives you an object which resolves methods using only the part of the MRO which comes after C.<br>
So, super() works with the MRO of the class and not just base classes.<br>
- Two arguments are passed when we call super(), the class from where itself on MRO, so it can trim MRO to resolve method call.
- i.e. super() looks for MRO of class it is called from and start looking for methods starting from first base onwards.<br><br><br>
<b>So why SortedIntList worked the way it worked?</b><br>
Because super() doesn't just call the base class it calls the class next on MRO.<br>
- super() from SortedIntList will call IntList.
- Inside IntList __ init__ we again encounter super() but it will also look for the <b>next class on the MRO of SortedIntList</b> and not Intlist, i.e. super call from here will deligate __ init__ of SortedList, which is next on MRO.
- super() from SortedList will deligate SimpleList because that is the next class on <b>MRO of SortedIntList</b>

If super() is called from any method of an instance it will make use of the self argument of that method to determine the class and thus the MRO for super() execution.<br>
But what if we call super() from class method? It doesn't have self argument and thus it will use cls argument.
### Class-bound Super Proxies

In [20]:
class Animal:
    @classmethod
    def description(cls):
        return "An animal"

class Bird(Animal):
    @classmethod
    def description(cls):
        s = super()
        print(s)
        print(s.description)
        return s.description() + " with wings"

class Flamingo(Bird):
    @classmethod
    def description(cls):
        return super().description() + " and fabulous pink feathers"


In [21]:
Flamingo.__mro__

(__main__.Flamingo, __main__.Bird, __main__.Animal, object)

In [22]:
Flamingo.description()

<super: <class 'Bird'>, <Flamingo object>>
<bound method Animal.description of <class '__main__.Flamingo'>>


'An animal with wings and fabulous pink feathers'

We can explicitely pass arguments to super(class_object, instance_or_class)
- The first argument will tell where to trim the MRO while second argument will be used to get the MRO.

In [23]:
s = SortedIntList()
# We can overpass the int validation by trimming it from SortedIntList MRO
# Use MRO of S -- SortedIntList, starting from IntList(excluded).
# If we didn't specified first argument would have been SortedIntList
super(IntList, s).add('Now string can be added as we skipped validation of IntList class')
s

SortedIntList(['Now string can be added as we skipped validation of IntList class'])